In [1]:
import numpy as np
import pandas as pd
import timeit

In [28]:
df_train = pd.read_csv("SSTB_train.csv")
df_train

,Unnamed: 0,sentence_index,sentence,splitset_label,id,values
0,0,1,The Rock is destined to be the 21st Century 's...,1,226166,0.69444
1,1,2,The gorgeously elaborate continuation of `` Th...,1,226300,0.83333
2,2,61,Singer\/composer Bryan Adams contributes a sle...,1,225801,0.62500
3,3,62,You 'd think by now America would have had eno...,1,14646,0.50000
4,4,63,Yet the act is still charming here .,1,14644,0.72222
...,...,...,...,...,...,...
8539,8539,11851,A real snooze .,1,222071,0.11111
8540,8540,11852,No surprises .,1,225165,0.22222
8541,8541,11853,We 've seen the hippie-turned-yuppie plot befo...,1,226985,0.75000
8542,8542,11854,Her fans walked out muttering words like `` ho...,1,223632,0.13889


In [29]:
df_test = pd.read_csv("SSTB_test.csv")
df_test

,Unnamed: 0,sentence_index,sentence,splitset_label,id,values
0,0,3,Effective but too-tepid biopic,2,13995,0.513890
1,1,4,If you sometimes like to go to the movies to h...,2,14123,0.736110
2,2,5,"Emerges as something rare , an issue movie tha...",2,13999,0.861110
3,3,6,The film provides some great insight into the ...,2,14498,0.597220
4,4,7,Offers that rare combination of entertainment ...,2,14351,0.833330
...,...,...,...,...,...,...
2205,2205,11621,An imaginative comedy\/thriller .,2,13851,0.777780
2206,2206,11623,"( A ) rare , beautiful film .",2,18182,0.916670
2207,2207,11626,( An ) hilarious romantic comedy .,2,23211,0.888890
2208,2208,11628,Never ( sinks ) into exploitation .,2,26177,0.625000


In [30]:
X_train = df_train["sentence"]
y_train = df_train["values"]

X_test = df_test["sentence"]
y_test = df_test["values"]

In [31]:
# function for converting the y values to positive or negative values

def convert_to_pos_neg(y):
    for i in range(len(y)):
        if y[i] > 0.5:
            y[i] = 1
        else:
            y[i] = 0
    return y

In [45]:
def convert_to_fine_grain(y):
    for i in range(len(y)):
        if y[i] >= 0 and y[i] <= 0.2:
            y[i] = 0
        elif y[i] > 0.2 and y[i] < 0.4:
            y[i] = 1
        elif y[i] > 0.4 and y[i] < 0.6:
            y[i] = 2
        elif y[i] > 0.6 and y[i] < 0.8:
            y[i] = 3
        elif y[i] > 0.8 and y[i] <= 1.0:
            y[i] = 4
    return y

In [ ]:
#====================Convert y into positive / negative==================

In [46]:
y_train_posneg = np.copy(y_train)  # make sure to use the copied value in order to prevent from modifying the original y values
y_train_posneg = convert_to_pos_neg(y_train_posneg)
y_train_posneg

array([1., 1., 1., ..., 1., 0., 0.])

In [47]:
y_test_posneg = np.copy(y_test)
y_test_posneg = convert_to_pos_neg(y_test_posneg)
y_test_posneg

array([1., 1., 1., ..., 1., 1., 0.])

In [48]:
#=====================================================================

In [49]:
#======================Convert y into fine grain values========================

In [50]:
y_train_finegrain = np.copy(y_train)
y_train_finegrain = convert_to_fine_grain(y_train_finegrain)
y_train_finegrain

array([3., 4., 3., ..., 3., 0., 1.])

In [51]:
y_test_finegrain = np.copy(y_test)
y_test_finegrain = convert_to_fine_grain(y_test_finegrain)
y_test_finegrain

array([2., 3., 4., ..., 4., 3., 0.])

In [84]:
#===============================================================

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.metrics import accuracy_score

In [54]:
cvec = CountVectorizer()
tfidf = TfidfVectorizer()

X_train_cvec = cvec.fit_transform(X_train, y_train)
X_train_tf = tfidf.fit_transform(X_train, y_train)

X_test_cvec = cvec.transform(X_test)
X_test_tf = tfidf.transform(X_test)

In [55]:
#=====================Generic 5-fold Cross Validation Score On Classifying Positive/Negative==================
# This cross validation will only operate on the training set
# Start the Timer
start = timeit.default_timer()

mnb = MultinomialNB()
results_cv = model_selection.cross_val_score(mnb, X_train_cvec, y_train_posneg, cv = 5)
print("Cross validation with countvectorizer: ", results_cv.mean())

results_tf = model_selection.cross_val_score(mnb, X_train_tf, y_train_posneg, cv = 5)
print("Cross validation with TfidfVectorizer: ", results_tf.mean())
# mnb.fit(X_train_tf, y_train)
# y_pred = mnb.predict(X_test_tf)

# print(accuracy_score(y_pred, y_test))

# Timer stops
stop = timeit.default_timer()
print("Time Execution: {}".format(stop - start))

Cross validation with countvectorizer:  0.7456616918558999
Cross validation with TfidfVectorizer:  0.751045642095916
Time Execution: 0.1225414880000244


In [56]:
#=====================Generic 5-fold Cross Validation Score on Classifying Fine Grain==================
# This cross validation will only operate on the training set
# Start the Timer
start = timeit.default_timer()

mnb = MultinomialNB()
results_cv = model_selection.cross_val_score(mnb, X_train_cvec, y_train_finegrain, cv = 5)
print("Cross validation with countvectorizer: ", results_cv.mean())

results_tf = model_selection.cross_val_score(mnb, X_train_tf, y_train_finegrain, cv = 5)
print("Cross validation with TfidfVectorizer: ", results_tf.mean())
# mnb.fit(X_train_tf, y_train)
# y_pred = mnb.predict(X_test_tf)

# print(accuracy_score(y_pred, y_test))

# Timer stops
stop = timeit.default_timer()
print("Time Execution: {}".format(stop - start))

Cross validation with countvectorizer:  0.3959462336233364
Cross validation with TfidfVectorizer:  0.39465968487991576
Time Execution: 0.12854977300003156


In [ ]:
#===========================Positive/Negative Classification=========================

In [57]:
#============================Finding Optimal N_gram=============================

In [63]:
# function for evaluating the accuracy of a given classifier and vectorizer
def nfeature_accuracy_checker(X_train, y_train, X_test, y_test, vectorizer=None, n_features=None, stop_words=None, ngram_range=(1, 1), classifier=None):
    result = []
    print (classifier)
    print ("\n")
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        t0 = time()
        sentiment_fit = checker_pipeline.fit(X_train, y_train)
        y_pred = sentiment_fit.predict(X_test)
        train_test_time = time() - t0
        accuracy = accuracy_score(y_pred, y_test)
        print("accuracy with", n, "features:", accuracy)
    print("\n")

In [64]:
from sklearn.pipeline import Pipeline
from time import time

In [65]:
cvec = CountVectorizer()
tfidf = TfidfVectorizer()
mnb = MultinomialNB()

In [66]:
stop_word_extractor = [None, "english"]
vectorizer = [cvec, tfidf]

In [67]:
n_features = np.arange(10000,50001,10000)

# unigram case 
for i in range(len(stop_word_extractor)):
    if i == 0:
        print("==============================INCLUDING STOP WORDS============================")
    else:
        print("==============================NOT INCLUDING STOP WORDS============================")
    for j in range(len(vectorizer)):
        if j == 0:
            print("\t=================COUNTVECTORIZER===============")
        else:
            print("\t=================TFIDF=============")
        feature_result_unigram = nfeature_accuracy_checker(X_train, y_train_posneg, X_test, y_test_posneg,
                                                                                   vectorizer=vectorizer[j], n_features=n_features, stop_words=stop_word_extractor[i], classifier=mnb)

==============================INCLUDING STOP WORDS============================
	=================COUNTVECTORIZER===============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 10000 features: 0.7701357466063349
accuracy with 20000 features: 0.7737556561085973
accuracy with 30000 features: 0.7737556561085973
accuracy with 40000 features: 0.7737556561085973
accuracy with 50000 features: 0.7737556561085973


	=================TFIDF=============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 10000 features: 0.7755656108597285
accuracy with 20000 features: 0.7782805429864253
accuracy with 30000 features: 0.7782805429864253
accuracy with 40000 features: 0.7782805429864253
accuracy with 50000 features: 0.7782805429864253


==============================NOT INCLUDING STOP WORDS============================
	=================COUNTVECTORIZER===============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 10000 featur

In [69]:
# unigram + bigram case 
# in here, we modify the number of features from the range 30000 to 130000, since 
# much more features will be created

n_features = np.arange(30000,130001,10000)

for i in range(len(stop_word_extractor)):
    if i == 0:
        print("==============================INCLUDING STOP WORDS============================")
    else:
        print("==============================NOT INCLUDING STOP WORDS============================")
    for j in range(len(vectorizer)):
        if j == 0:
            print("\t=================COUNTVECTORIZER===============")
        else:
            print("\t=================TFIDF=============")
        feature_result_unigram = nfeature_accuracy_checker(X_train, y_train_posneg, X_test, y_test_posneg,
                                                                vectorizer=vectorizer[j], n_features=n_features, stop_words=stop_word_extractor[i], ngram_range = (1, 2), classifier=mnb)

==============================INCLUDING STOP WORDS============================
	=================COUNTVECTORIZER===============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 30000 features: 0.7760180995475113
accuracy with 40000 features: 0.7755656108597285
accuracy with 50000 features: 0.7760180995475113
accuracy with 60000 features: 0.7773755656108597
accuracy with 70000 features: 0.7782805429864253
accuracy with 80000 features: 0.7782805429864253
accuracy with 90000 features: 0.781447963800905
accuracy with 100000 features: 0.7823529411764706
accuracy with 110000 features: 0.7823529411764706
accuracy with 120000 features: 0.7823529411764706
accuracy with 130000 features: 0.7823529411764706


	=================TFIDF=============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 30000 features: 0.7778280542986425
accuracy with 40000 features: 0.7841628959276018
accuracy with 50000 features: 0.781447963800905
accuracy with 60000 fea

In [70]:
# unigram + bigram + trigram case
# We furthur extent the number of features to the range of 120000

n_features = np.arange(60000,160001,10000)

for i in range(len(stop_word_extractor)):
    if i == 0:
        print("==============================INCLUDING STOP WORDS============================")
    else:
        print("==============================NOT INCLUDING STOP WORDS============================")
    for j in range(len(vectorizer)):
        if j == 0:
            print("\t=================COUNTVECTORIZER===============")
        else:
            print("\t=================TFIDF=============")
        feature_result_unigram = nfeature_accuracy_checker(X_train, y_train_posneg, X_test, y_test_posneg,
                                                                vectorizer=vectorizer[j], n_features=n_features, stop_words=stop_word_extractor[i], ngram_range = (1, 3), classifier=mnb)

==============================INCLUDING STOP WORDS============================
	=================COUNTVECTORIZER===============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 60000 features: 0.7746606334841629
accuracy with 70000 features: 0.7737556561085973
accuracy with 80000 features: 0.7760180995475113
accuracy with 90000 features: 0.7723981900452489
accuracy with 100000 features: 0.7714932126696833
accuracy with 110000 features: 0.7733031674208145
accuracy with 120000 features: 0.7719457013574661
accuracy with 130000 features: 0.7719457013574661
accuracy with 140000 features: 0.7737556561085973
accuracy with 150000 features: 0.7733031674208145
accuracy with 160000 features: 0.7760180995475113


	=================TFIDF=============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 60000 features: 0.7782805429864253
accuracy with 70000 features: 0.7746606334841629
accuracy with 80000 features: 0.7778280542986425
accuracy with 9000

In [71]:
# So far, the bigram case with CountVectorizer 70000 features has the best result

In [72]:
mnbGV = MultinomialNB()


cvecGV = CountVectorizer(ngram_range = (1, 2), max_features = 70000)
# tvecGV = TfidfVectorizer(ngram_range = (1, 2), max_features = 100000, stop_words = "english")
# tvecGV = TfidfVectorizer(ngram_range = (1, 2), max_features = 100000, analyzer = lemm_analyzer)
X_cvec = cvecGV.fit_transform(X_train)

In [98]:
#-----------------------------GRID SEARCH CROSS VALIDATION------------------------------
from sklearn.model_selection import GridSearchCV

# Timer begins
start = timeit.default_timer()

tuned_parameters = [{'alpha' : [1.5, 1.7, 1.75, 1.8, 2.0]}]
n_folds = 5

grid_search = GridSearchCV(estimator = mnb, param_grid = tuned_parameters, cv = n_folds, refit = False, n_jobs = -1)

grid_search.fit(X_cvec, y_train_posneg)

scores = grid_search.cv_results_['mean_test_score']
scores_std = grid_search.cv_results_['std_test_score']
print('scores:',scores)
print('scores_std',scores_std)

# Optimal hyperparameter
bestAlpha = grid_search.best_params_['alpha']
print(grid_search.best_params_)

# Timer stops
stop = timeit.default_timer()
print("Time Execution: {}".format(stop - start))
# #-----------------------------END OF GRID SEARCH-----------------------------------------

scores: [0.75632022 0.75784176 0.75772472 0.75760768 0.75632022]
scores_std [0.03872693 0.03740403 0.03728401 0.03769677 0.03742853]
{'alpha': 1.7}
Time Execution: 0.3508574940001381


In [110]:
def hyper_parameter_tuning(alphas):
    best_alpha = None
    best_result = 0
    
    for a in alphas:
        mnb_model = MultinomialNB(alpha = a)
        X_final = cvecGV.transform(X_test)
        mnb_model.fit(X_cvec, y_train_posneg)
        y_pred = mnb_model.predict(X_final)
        result = accuracy_score(y_pred, y_test_posneg)
        print("accuracy for alpha =", a, ":", result)
        if result > best_result:
            best_result = result
            best_alpha = a
        
    print("The best result is",best_result,"with alpha of", best_alpha)

In [114]:
alphas = [2.5, 2.6, 2.7, 2.8, 2.9, 3.0]
hyper_parameter_tuning(alphas)

accuracy for alpha = 2.5 : 0.781447963800905
accuracy for alpha = 2.6 : 0.7828054298642534
accuracy for alpha = 2.7 : 0.7828054298642534
accuracy for alpha = 2.8 : 0.7823529411764706
accuracy for alpha = 2.9 : 0.7819004524886878
accuracy for alpha = 3.0 : 0.7809954751131222
The best result is 0.7828054298642534 with alpha of 2.6


In [120]:
# The best hyperparameter value so far is 2.6

In [ ]:
#====================Bagging Classifier with NB Model================

In [122]:
from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection

# Timer begins
# start = timeit.default_timer()


In [129]:
mnb = MultinomialNB(alpha = 2.6)
X_final = cvecGV.transform(X_test)

In [130]:
def bagging_max_sample_optimization(samples):
    best_sample = 0
    best_result = 0
    
    for s in samples:
        bg = BaggingClassifier(mnb, max_samples = s, max_features = 0.5, n_estimators = 200)
        bg.fit(X_cvec, y_train_posneg)
        y_pred = bg.predict(X_final)
        result = accuracy_score(y_pred, y_test_posneg)
        print("Bagging Classifier with {} ratio of sample is {}".format(s, result))
        if result > best_result:
            best_result = result
            best_sample = s
    print("The best result is {} ratio of sample with accuracy of {}".format(best_sample, best_result))

In [137]:
samples = [0.6, 0.7, 0.8, 0.9, 1.0]
bagging_max_sample_optimization(samples)

Bagging Classifier with 0.6 ratio of sample is 0.7773755656108597
Bagging Classifier with 0.7 ratio of sample is 0.779185520361991
Bagging Classifier with 0.8 ratio of sample is 0.7760180995475113
Bagging Classifier with 0.9 ratio of sample is 0.7796380090497738
Bagging Classifier with 1.0 ratio of sample is 0.7764705882352941
The best result is 0.9 ratio of sample with accuracy of 0.7796380090497738


In [138]:
# The best ratio is so far is 0.9, so we set the best sample ratio as a variable
bestSampleRate = 0.9

In [139]:
def bagging_max_feature_optimization(features):
    best_feature = 0
    best_result = 0
    
    for f in features:
        bg = BaggingClassifier(mnb, max_samples = bestSampleRate, max_features = f, n_estimators = 200)
        bg.fit(X_cvec, y_train_posneg)
        y_pred = bg.predict(X_final)
        result = accuracy_score(y_pred, y_test_posneg)
        print("Bagging Classifier with {} ratio of sample is {}".format(f, result))
        if result > best_result:
            best_result = result
            best_feature = f
    print("The best result is {} ratio of sample with accuracy of {}".format(best_feature, best_result))

In [142]:
features = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
bagging_max_feature_optimization(features)

# # Timer stops
# stop = timeit.default_timer()
# print("Time Execution bagging classifier: {}".format(stop - start))

Bagging Classifier with 0.4 ratio of sample is 0.7782805429864253
Bagging Classifier with 0.5 ratio of sample is 0.7778280542986425
Bagging Classifier with 0.6 ratio of sample is 0.7832579185520362
Bagging Classifier with 0.7 ratio of sample is 0.7823529411764706
Bagging Classifier with 0.8 ratio of sample is 0.7769230769230769
Bagging Classifier with 0.9 ratio of sample is 0.7832579185520362
The best result is 0.6 ratio of sample with accuracy of 0.7832579185520362


In [144]:
bestFeatureRatio = 0.6

In [148]:
def bagging_max_estimator_optimization(estimators):
    best_estim = 0
    best_result = 0
    
    for e in estimators:
        bg = BaggingClassifier(mnb, max_samples = bestSampleRate, max_features = bestFeatureRatio, n_estimators = e)
        bg.fit(X_cvec, y_train_posneg)
        y_pred = bg.predict(X_final)
        result = accuracy_score(y_pred, y_test_posneg)
        print("Bagging Classifier with {} ratio of sample is {}".format(e, result))
        if result > best_result:
            best_result = result
            best_estim = e
    print("The best result is {} estimators with accuracy of {}".format(best_estim, best_result))

In [151]:
estimators = [400, 600, 800, 1000]
bagging_max_estimator_optimization(estimators)

Bagging Classifier with 400 ratio of sample is 0.7796380090497738
Bagging Classifier with 600 ratio of sample is 0.7796380090497738
Bagging Classifier with 800 ratio of sample is 0.7819004524886878
Bagging Classifier with 1000 ratio of sample is 0.7809954751131222
The best result is 800 estimators with accuracy of 0.7819004524886878


In [121]:
#==============================================================

In [152]:
#===================================End of Positive / Negative Section==================================

In [153]:
#==================================Fine Grain Section===============================================

In [154]:
cvec = CountVectorizer()
tfidf = TfidfVectorizer()
mnb = MultinomialNB()

In [155]:
stop_word_extractor = [None, "english"]
vectorizer = [cvec, tfidf]

In [167]:
n_features = np.arange(2500,20001,2500)

# unigram case 
for i in range(len(stop_word_extractor)):
    if i == 0:
        print("==============================INCLUDING STOP WORDS============================")
    else:
        print("==============================NOT INCLUDING STOP WORDS============================")
    for j in range(len(vectorizer)):
        if j == 0:
            print("\t=================COUNTVECTORIZER===============")
        else:
            print("\t=================TFIDF=============")
        feature_result_unigram = nfeature_accuracy_checker(X_train, y_train_finegrain, X_test, y_test_finegrain,
                                                                                   vectorizer=vectorizer[j], n_features=n_features, stop_words=stop_word_extractor[i], classifier=mnb)

==============================INCLUDING STOP WORDS============================
	=================COUNTVECTORIZER===============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 2500 features: 0.41266968325791853
accuracy with 5000 features: 0.4171945701357466
accuracy with 7500 features: 0.4108597285067873
accuracy with 10000 features: 0.4072398190045249
accuracy with 12500 features: 0.4076923076923077
accuracy with 15000 features: 0.4113122171945701
accuracy with 17500 features: 0.4099547511312217
accuracy with 20000 features: 0.4099547511312217


	=================TFIDF=============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 2500 features: 0.39819004524886875
accuracy with 5000 features: 0.40361990950226245
accuracy with 7500 features: 0.40180995475113124
accuracy with 10000 features: 0.3995475113122172
accuracy with 12500 features: 0.3990950226244344
accuracy with 15000 features: 0.39728506787330314
accuracy with 17500 featur

In [168]:
# unigram + bigram case 
# in here, we modify the number of features from the range 30000 to 130000, since 
# much more features will be created

n_features = np.arange(2500,25001,2500)

for i in range(len(stop_word_extractor)):
    if i == 0:
        print("==============================INCLUDING STOP WORDS============================")
    else:
        print("==============================NOT INCLUDING STOP WORDS============================")
    for j in range(len(vectorizer)):
        if j == 0:
            print("\t=================COUNTVECTORIZER===============")
        else:
            print("\t=================TFIDF=============")
        feature_result_unigram = nfeature_accuracy_checker(X_train, y_train_finegrain, X_test, y_test_finegrain,
                                                                vectorizer=vectorizer[j], n_features=n_features, stop_words=stop_word_extractor[i], ngram_range = (1, 2), classifier=mnb)

==============================INCLUDING STOP WORDS============================
	=================COUNTVECTORIZER===============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 2500 features: 0.38823529411764707
accuracy with 5000 features: 0.39728506787330314
accuracy with 7500 features: 0.41266968325791853
accuracy with 10000 features: 0.4085972850678733
accuracy with 12500 features: 0.4095022624434389
accuracy with 15000 features: 0.4072398190045249
accuracy with 17500 features: 0.40407239819004526
accuracy with 20000 features: 0.4072398190045249
accuracy with 22500 features: 0.4095022624434389
accuracy with 25000 features: 0.4117647058823529


	=================TFIDF=============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 2500 features: 0.3832579185520362
accuracy with 5000 features: 0.38868778280542987
accuracy with 7500 features: 0.3904977375565611
accuracy with 10000 features: 0.3891402714932127
accuracy with 12500 featur

In [166]:
# unigram + bigram + trigram case
# We furthur extent the number of features to the range of 120000

n_features = np.arange(20000,70001,5000)

for i in range(len(stop_word_extractor)):
    if i == 0:
        print("==============================INCLUDING STOP WORDS============================")
    else:
        print("==============================NOT INCLUDING STOP WORDS============================")
    for j in range(len(vectorizer)):
        if j == 0:
            print("\t=================COUNTVECTORIZER===============")
        else:
            print("\t=================TFIDF=============")
        feature_result_unigram = nfeature_accuracy_checker(X_train, y_train_finegrain, X_test, y_test_finegrain,
                                                                vectorizer=vectorizer[j], n_features=n_features, stop_words=stop_word_extractor[i], ngram_range = (1, 3), classifier=mnb)

==============================INCLUDING STOP WORDS============================
	=================COUNTVECTORIZER===============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 20000 features: 0.40180995475113124
accuracy with 25000 features: 0.4108597285067873
accuracy with 30000 features: 0.41312217194570133
accuracy with 35000 features: 0.4063348416289593
accuracy with 40000 features: 0.40361990950226245
accuracy with 45000 features: 0.3986425339366516
accuracy with 50000 features: 0.40135746606334843
accuracy with 55000 features: 0.3995475113122172
accuracy with 60000 features: 0.4
accuracy with 65000 features: 0.4004524886877828
accuracy with 70000 features: 0.3995475113122172


	=================TFIDF=============
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


accuracy with 20000 features: 0.3904977375565611
accuracy with 25000 features: 0.38868778280542987
accuracy with 30000 features: 0.3891402714932127
accuracy with 35000 features: 0.391

In [172]:
# So far, the unigram case with CountVectorizer 5000 features has the best result

In [173]:
mnbGV = MultinomialNB()


cvecGV = CountVectorizer(ngram_range = (1, 1), max_features = 5000)
# tvecGV = TfidfVectorizer(ngram_range = (1, 2), max_features = 100000, stop_words = "english")
# tvecGV = TfidfVectorizer(ngram_range = (1, 2), max_features = 100000, analyzer = lemm_analyzer)
X_cvec = cvecGV.fit_transform(X_train)

In [174]:
def hyper_parameter_tuning_fg(alphas):
    best_alpha = None
    best_result = 0
    
    for a in alphas:
        mnb_model = MultinomialNB(alpha = a)
        X_final = cvecGV.transform(X_test)
        mnb_model.fit(X_cvec, y_train_finegrain)
        y_pred = mnb_model.predict(X_final)
        result = accuracy_score(y_pred, y_test_finegrain)
        print("accuracy for alpha =", a, ":", result)
        if result > best_result:
            best_result = result
            best_alpha = a
        
    print("The best result is",best_result,"with alpha of", best_alpha)

In [178]:
alphas = [0.2, 0.3, 0.4, 0.5, 0.6]
hyper_parameter_tuning_fg(alphas)

accuracy for alpha = 0.2 : 0.4108597285067873
accuracy for alpha = 0.3 : 0.416289592760181
accuracy for alpha = 0.4 : 0.41990950226244345
accuracy for alpha = 0.5 : 0.42036199095022625
accuracy for alpha = 0.6 : 0.42036199095022625
The best result is 0.42036199095022625 with alpha of 0.5


In [179]:
# So far, the best alpha is 0.5

In [180]:
mnb = MultinomialNB(alpha = 0.5)
X_final = cvecGV.transform(X_test)

In [181]:
def bagging_max_sample_optimization_fg(samples):
    best_sample = 0
    best_result = 0
    
    for s in samples:
        bg = BaggingClassifier(mnb, max_samples = s, max_features = 0.5, n_estimators = 200)
        bg.fit(X_cvec, y_train_finegrain)
        y_pred = bg.predict(X_final)
        result = accuracy_score(y_pred, y_test_finegrain)
        print("Bagging Classifier with {} ratio of sample is {}".format(s, result))
        if result > best_result:
            best_result = result
            best_sample = s
    print("The best result is {} ratio of sample with accuracy of {}".format(best_sample, best_result))

In [183]:
features = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
bagging_max_sample_optimization_fg(features)

Bagging Classifier with 0.4 ratio of sample is 0.4076923076923077
Bagging Classifier with 0.5 ratio of sample is 0.4158371040723982
Bagging Classifier with 0.6 ratio of sample is 0.4122171945701357
Bagging Classifier with 0.7 ratio of sample is 0.4113122171945701
Bagging Classifier with 0.8 ratio of sample is 0.41809954751131223
Bagging Classifier with 0.9 ratio of sample is 0.416289592760181
The best result is 0.8 ratio of sample with accuracy of 0.41809954751131223


In [185]:
# The best sample ratio was 0.8
bestSampleRatioFG = 0.8

In [186]:
def bagging_max_feature_optimization_fg(features):
    best_feature = 0
    best_result = 0
    
    for f in features:
        bg = BaggingClassifier(mnb, max_samples = bestSampleRatioFG, max_features = f, n_estimators = 200)
        bg.fit(X_cvec, y_train_finegrain)
        y_pred = bg.predict(X_final)
        result = accuracy_score(y_pred, y_test_finegrain)
        print("Bagging Classifier with {} ratio of sample is {}".format(f, result))
        if result > best_result:
            best_result = result
            best_feature = f
    print("The best result is {} ratio of sample with accuracy of {}".format(best_feature, best_result))

In [187]:
samples = [0.6, 0.7, 0.8, 0.9, 1.0]
bagging_max_sample_optimization_fg(samples)

Bagging Classifier with 0.6 ratio of sample is 0.4158371040723982
Bagging Classifier with 0.7 ratio of sample is 0.41855203619909503
Bagging Classifier with 0.8 ratio of sample is 0.4171945701357466
Bagging Classifier with 0.9 ratio of sample is 0.41945701357466064
Bagging Classifier with 1.0 ratio of sample is 0.41809954751131223
The best result is 0.9 ratio of sample with accuracy of 0.41945701357466064


In [188]:
# The best feature ratio so far is 0.9
bestFeatureRatioFG = 0.9

In [189]:
def bagging_max_estimator_optimization(estimators):
    best_estim = 0
    best_result = 0
    
    for e in estimators:
        bg = BaggingClassifier(mnb, max_samples = bestSampleRatioFG, max_features = bestFeatureRatioFG, n_estimators = e)
        bg.fit(X_cvec, y_train_finegrain)
        y_pred = bg.predict(X_final)
        result = accuracy_score(y_pred, y_test_finegrain)
        print("Bagging Classifier with {} ratio of sample is {}".format(e, result))
        if result > best_result:
            best_result = result
            best_estim = e
    print("The best result is {} estimators with accuracy of {}".format(best_estim, best_result))

In [190]:
estimators = [400, 600, 800, 1000]
bagging_max_estimator_optimization(estimators)

Bagging Classifier with 400 ratio of sample is 0.41809954751131223
Bagging Classifier with 600 ratio of sample is 0.42036199095022625
Bagging Classifier with 800 ratio of sample is 0.41900452488687784
Bagging Classifier with 1000 ratio of sample is 0.42171945701357466
The best result is 1000 estimators with accuracy of 0.42171945701357466
